# Module 4: Feature Selection & Validation - Consolidated

**One notebook to rule them all.**

This runs the complete pipeline:
1. Feature extraction & selection
2. Model training (LASSO or Random Forest)
3. LOO cross-validation
4. Validation testing
5. Results visualization

**Runtime:** ~2 minutes per model

## Setup

In [1]:
import sys
from pathlib import Path

# Import the pipeline
from src.feature_selection import run_pipeline

import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image, display

print("✓ Ready to go!")

✓ Ready to go!


## Option 1: Run Random Forest

Random Forest may handle batch effects better than LASSO.

In [2]:
# Run Random Forest pipeline
rf_results = run_pipeline(model_type='rf')


MODULE 4: RF CLASSIFIER

LOADING DATA

Discovery: 8 samples (4 ALS, 4 Control)
Validation: 14 samples (8 ALS, 6 Control)

EXTRACTING FEATURES

Aggregating methylation to 500kb bins...

Fragmentomics: 17 → 17
Methylation: 80 → 74
Total features: 91

FEATURE SELECTION: RANDOM FOREST

Selected 20 features by importance
  - meth_agg_40: 0.0670
  - meth_agg_25: 0.0548
  - meth_agg_60: 0.0414
  - meth_agg_49: 0.0393
  - meth_agg_68: 0.0314
  - meth_agg_38: 0.0258
  - meth_agg_78: 0.0249
  - meth_agg_36: 0.0242
  - meth_agg_16: 0.0228
  - meth_agg_63: 0.0225
  - meth_agg_50: 0.0224
  - meth_agg_30: 0.0194
  - meth_agg_85: 0.0177
  - frag_pct_mononucleosomal: 0.0156
  - meth_agg_55: 0.0152
  - meth_agg_15: 0.0151
  - meth_agg_42: 0.0145
  - meth_agg_32: 0.0139
  - meth_agg_77: 0.0137
  - meth_agg_17: 0.0130

TRAINING RF WITH LOO-CV

Training: 8 samples × 20 features

LOO-CV AUC: 0.688

VALIDATION TESTING

Validation AUC: 0.187
Validation Accuracy: 0.500

Discovery LOO-CV: 0.688
Difference: -0

In [3]:
# Try fragmentomics-only approach
print("\n" + "="*70)
print("TRYING FRAGMENTOMICS-ONLY (no methylation)")
print("="*70)

from src.feature_selection import (
    load_data, extract_features, train_with_loo, validate
)

# Load data
discovery_df, validation_df = load_data()

# Extract features
feature_df, all_features, meth_agg_df = extract_features(discovery_df)

# Get ONLY fragmentomics features
frag_only = [f for f in all_features if not f.startswith('meth_agg_')]

print(f"\nUsing {len(frag_only)} fragmentomics features only")
for f in frag_only:
    print(f"  - {f}")

# Train with LOO-CV
model_data, loo_pred, loo_true = train_with_loo(
    feature_df, frag_only, model_type='rf'
)

# Validate
val_predictions, val_auc, val_acc = validate(
    validation_df, meth_agg_df, model_data
)

print("\n" + "="*70)
print("FRAGMENTOMICS-ONLY RESULT")
print("="*70)
print(f"Discovery LOO-CV: {model_data['loo_auc']:.3f}")
print(f"Validation AUC: {val_auc:.3f}")
print(f"Drop: {model_data['loo_auc'] - val_auc:.3f}")

if val_auc >= 0.60:
    print("\n✓ Better! Fragmentomics are more robust to batch effects")
else:
    print("\n✗ Still poor - fundamental batch effect problem")


TRYING FRAGMENTOMICS-ONLY (no methylation)

LOADING DATA

Discovery: 8 samples (4 ALS, 4 Control)
Validation: 14 samples (8 ALS, 6 Control)

EXTRACTING FEATURES

Aggregating methylation to 500kb bins...

Fragmentomics: 17 → 17
Methylation: 80 → 74
Total features: 91

Using 17 fragmentomics features only
  - frag_cv
  - frag_iqr
  - frag_kurtosis
  - frag_mean
  - frag_median
  - frag_pct_dinucleosomal
  - frag_pct_long
  - frag_pct_mononucleosomal
  - frag_pct_short
  - frag_pct_very_short
  - frag_q25
  - frag_q50
  - frag_q75
  - frag_ratio_mono_di
  - frag_ratio_short_long
  - frag_skewness
  - frag_std

TRAINING RF WITH LOO-CV

Training: 8 samples × 17 features

LOO-CV AUC: 0.125

VALIDATION TESTING

Validation AUC: 0.667
Validation Accuracy: 0.571

Discovery LOO-CV: 0.125
Difference: +0.542

⚠ MODERATE: Weak generalization

FRAGMENTOMICS-ONLY RESULT
Discovery LOO-CV: 0.125
Validation AUC: 0.667
Drop: -0.542

✓ Better! Fragmentomics are more robust to batch effects


In [4]:
# Simple approach: Just use the most biologically interpretable features
print("\n" + "="*70)
print("SIMPLE BIOLOGICAL APPROACH")
print("="*70)

# Just use 5 core fragment size features that should be robust
simple_features = [
    'frag_mean',           # Overall fragment size
    'frag_pct_short',      # Short fragments (nucleosome-free)
    'frag_pct_long',       # Long fragments  
    'frag_ratio_short_long', # Ratio of short to long
    'frag_pct_mononucleosomal' # Mononucleosome peak
]

print(f"\nUsing {len(simple_features)} biologically interpretable features:")
for f in simple_features:
    print(f"  - {f}")

# Train
model_data_simple, loo_pred, loo_true = train_with_loo(
    feature_df, simple_features, model_type='rf'
)

# Validate
val_predictions_simple, val_auc_simple, val_acc_simple = validate(
    validation_df, meth_agg_df, model_data_simple
)

print("\n" + "="*70)
print("SIMPLE FEATURES RESULT")
print("="*70)
print(f"Discovery LOO-CV: {model_data_simple['loo_auc']:.3f}")
print(f"Validation AUC: {val_auc_simple:.3f}")
print(f"Validation Accuracy: {val_acc_simple:.3f}")

if val_auc_simple >= 0.70:
    print("\n🎉 SUCCESS! Simple features work better!")
    print("\nThis makes sense because:")
    print("  - Core fragment metrics are well-established in cfDNA")
    print("  - They're less prone to technical batch effects")
    print("  - Simpler model = less overfitting")


SIMPLE BIOLOGICAL APPROACH

Using 5 biologically interpretable features:
  - frag_mean
  - frag_pct_short
  - frag_pct_long
  - frag_ratio_short_long
  - frag_pct_mononucleosomal

TRAINING RF WITH LOO-CV

Training: 8 samples × 5 features

LOO-CV AUC: 0.188

VALIDATION TESTING

Validation AUC: 0.646
Validation Accuracy: 0.643

Discovery LOO-CV: 0.188
Difference: +0.458

⚠ MODERATE: Weak generalization

SIMPLE FEATURES RESULT
Discovery LOO-CV: 0.188
Validation AUC: 0.646
Validation Accuracy: 0.643


### XGBOOST Results

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
import xgboost as xgb

print("\n" + "="*70)
print("TESTING XGBOOST")
print("="*70)

# Use same 5 simple features
simple_features = [
    'frag_mean', 'frag_pct_short', 'frag_pct_long',
    'frag_ratio_short_long', 'frag_pct_mononucleosomal'
]

X = feature_df[simple_features].fillna(feature_df[simple_features].median()).values
y = (feature_df['disease_status'] == 'als').astype(int).values

# XGBoost with aggressive regularization for n=8
xgb_params = {
    'max_depth': 2,              # Very shallow
    'learning_rate': 0.1,        # Conservative
    'n_estimators': 100,         # Fewer trees
    'min_child_weight': 2,       # Prevent overfitting
    'subsample': 0.8,            # Row sampling
    'colsample_bytree': 0.8,     # Feature sampling
    'reg_alpha': 1.0,            # L1 regularization
    'reg_lambda': 1.0,           # L2 regularization
    'random_state': 42,
    'eval_metric': 'logloss'
}

# LOO Cross-validation
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
loo_predictions = []
loo_true = []

for train_idx, test_idx in loo.split(X):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    # Calculate scale_pos_weight for this fold
    n_pos = y_train.sum()
    n_neg = len(y_train) - n_pos
    scale_pos_weight = n_neg / n_pos if n_pos > 0 else 1
    
    model = xgb.XGBClassifier(**xgb_params, scale_pos_weight=scale_pos_weight)
    model.fit(X_train, y_train, verbose=False)
    
    y_pred_proba = model.predict_proba(X_test)[0, 1]
    loo_predictions.append(y_pred_proba)
    loo_true.append(y_test[0])

loo_predictions = np.array(loo_predictions)
loo_true = np.array(loo_true)

from sklearn.metrics import roc_auc_score, accuracy_score
loo_auc = roc_auc_score(loo_true, loo_predictions)
print(f"\nXGBoost Discovery LOO-CV AUC: {loo_auc:.3f}")

# Train final model
final_xgb = xgb.XGBClassifier(**xgb_params)
final_xgb.fit(X, y, verbose=False)

# Validate
val_feature_df = validation_df[['sample_id', 'disease_status']].copy()
for feat in simple_features:
    val_feature_df[feat] = validation_df[feat]

X_val = val_feature_df[simple_features].fillna(val_feature_df[simple_features].median()).values
y_val = (validation_df['disease_status'] == 'als').astype(int).values

y_pred_proba_val = final_xgb.predict_proba(X_val)[:, 1]
y_pred_val = (y_pred_proba_val >= 0.5).astype(int)

val_auc_xgb = roc_auc_score(y_val, y_pred_proba_val)
val_acc_xgb = accuracy_score(y_val, y_pred_val)

print(f"XGBoost Validation AUC: {val_auc_xgb:.3f}")
print(f"XGBoost Validation Accuracy: {val_acc_xgb:.3f}")

print("\n" + "="*70)
print("COMPARISON: RF vs XGBoost")
print("="*70)
print(f"Random Forest    - Validation AUC: 0.646, Accuracy: 64.3%")
print(f"XGBoost          - Validation AUC: {val_auc_xgb:.3f}, Accuracy: {val_acc_xgb*100:.1f}%")

if val_auc_xgb > 0.646 + 0.05:
    print("\n🎉 XGBoost is better! Use this for final report.")
elif val_auc_xgb > 0.646:
    print("\n✓ XGBoost slightly better, but similar to RF")
else:
    print("\n→ RF and XGBoost perform similarly, stick with RF (simpler)")


TESTING XGBOOST

XGBoost Discovery LOO-CV AUC: 0.500
XGBoost Validation AUC: 0.500
XGBoost Validation Accuracy: 0.571

COMPARISON: RF vs XGBoost
Random Forest    - Validation AUC: 0.646, Accuracy: 64.3%
XGBoost          - Validation AUC: 0.500, Accuracy: 57.1%

→ RF and XGBoost perform similarly, stick with RF (simpler)
